In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import calendar as cl

In [ ]:
#Archivo que muestra la serie de tiempo de lo recaudado por país
serie_previa = pd.read_csv('archivo_dia_anterior.csv', index_col = 0)
#Archivo con la suma de lo recaudado en cada país, hasta el reporte del día anterior
suma_paises = pd.read_csv('archivo_suma_dia_anterior.csv', index_col = 0)

In [ ]:
previo = datetime.today() - timedelta(days = 1) #se resta un día para apilar el día previo al registro

#Diccionario que reprecenta el nuevo renglón a agregar
dict_ren = dict()
i = 0
for k in [*serie_previa]:
    i = i + 1
    dict_ren[k] = 0
    
indice = previo.strftime('%Y-%m-%d')
renglon_final = pd.DataFrame(dict_ren, index = [indice])
serie_previa = serie_previa.append(renglon_final)

In [ ]:
#Variables requeridas en todo el proceso de acumulación de data.
hoy = datetime.today()
fecha_ini = datetime(2021,12,14)
fecha_previa = datetime.strptime(serie_previa.index[-1], '%Y-%m-%d')  #Carga del día previo de registro
fecha_previa_str = serie_previa.index[-1]
rate_eua = 0.93813
rate_can = 1 - rate_eua

In [ ]:
archivo = open('the_numbers\\Numbers_no_way_home_12-26.html', encoding = 'utf8')
html_numbers = archivo.read()
archivo.close()
soup_numbers = BeautifulSoup(html_numbers)

#Se filtran los objetos que hay en la variable de soup, con el fin de identificar en qué grado podemos extraer la tabla
lista_busqueda = list()
for tab in soup_numbers('tr'):
    for a in tab:
        for b in a:
            lista_busqueda.append(b)
            
#Definición del ingreso en Candá y EUA en el último día
# y del ingreso internacional excepto Canadá y EUA
for i in range(len(lista_busqueda)):
    if str(lista_busqueda[i]) == '<a href="/box-office-chart/daily/2021/12/25">Dec 25, 2021</a>':
        cadena = str(lista_busqueda[i+4])
        cadena = cadena.replace('$','')
        cadena = cadena.replace(',','')
        domestic = int(cadena)
    if str(lista_busqueda[i]) == '<b>Rest of World</b>':
        cadena = str(lista_busqueda[i+2])
        cadena = cadena.replace('<b>','').replace('</b>','')
        cadena = cadena.replace('$','')
        cadena = cadena.replace(',','')
        resto_mundo = int(cadena)
    if str(lista_busqueda[i]) == '<b>International Total</b>':
        cadena = str(lista_busqueda[i+1])
        cadena = cadena.replace('<b>','').replace('</b>','')
        cadena = cadena.replace('$','')
        cadena = cadena.replace(',','')
        ingreso_inter = int(cadena)
        
#domestic = 0 #Esto se pone cuando no haya actualización del día anterior para EUA y Canadá       

ingreso_total = ingreso_inter + domestic + int(suma_paises.loc['United States']) + int(suma_paises.loc['Canada'])
    
for i in range(len(lista_busqueda)):
    if lista_busqueda[i] == 'Territory':
        print(lista_busqueda[i], i)
        ind1 = i
    if str(lista_busqueda[i]) == '<b>International Total</b>':
        print(lista_busqueda[i+3], i+3)
        ind2 = i+3

lista_busqueda = lista_busqueda[ind1:ind2+1]
#Para combrobar que se ha extraído lo correcto
print(len(lista_busqueda), domestic, ingreso_inter, resto_mundo, ingreso_total) 

In [ ]:
columnas = ['Territory', 'Release', 'Opening', 'Opening Theaters', 'Maximun Theaters',
            'Theatrical Engagement', 'Box Office', 'Report Date']

paises = list()
fechas = list()
i_estreno = list()
cines_estreno = list()
cines = list()
cine_eng = list()
ingresos = list()
f_reporte = list()

i = 0
for ele in lista_busqueda:
    i = i + 1
    c_aux = str(ele)
    if str(ele) == '<b>Rest of World</b>':
        continue
    elif str(ele) == '<b>International Total</b>':
        break
    if len(c_aux) > 15:
        for a in ele:
            for b in a:
                pais = str(b)
                if pais != '*':
                    paises.append(pais)
        j = i-1
        #Para conocer este patrón se hace una revisión visual de la lista_busqueda para verlo
        #Puede cambiar según se llene de data la tabla. 
        fechas.append(lista_busqueda[j+2])
        i_estreno.append(lista_busqueda[j+4])
        cines_estreno.append(lista_busqueda[j+6])
        cines.append(lista_busqueda[j+8])
        cine_eng.append(lista_busqueda[j+10])
        ingresos.append(lista_busqueda[j+12])
        f_reporte.append(lista_busqueda[j+14])
        
df_numbers = pd.DataFrame({columnas[0]:paises, columnas[1]:fechas, columnas[2]:i_estreno, columnas[3]:cines_estreno,
                          columnas[4]:cines, columnas[5]:cine_eng, columnas[6]:ingresos, columnas[7]:f_reporte})
df_numbers

In [ ]:
df_paises = df_numbers.drop(['Opening Theaters', 'Maximun Theaters', 'Theatrical Engagement'], axis = 'columns')
#df_paises incluye las fechas de reportes de los ingresos

#Aquí se limpia la moneda de forma que sólo se trabaje con valores enteros, y se calculan las distribuciones por día
dias_exhi = list()  #Dias desde el estreno
ingreso = list()
fecha_estreno = list()
ingreso_dia = list()

ind_rusia = df_paises[df_paises['Territory'] == 'Russia (CIS)'].index.item()
df_paises.loc[ind_rusia, 'Territory'] = 'Russia'

for ind in df_paises.index:
    
    f_estreno = datetime.strptime(df_paises.loc[ind, 'Release'],'%m/%d/%Y')
    fecha_e = f_estreno

    fecha_estreno.append(fecha_e)
    try:
        f_reporte = df_paises.loc[ind,'Report Date'].split('/')
        anio = int(f_reporte[2])
        mes = int(f_reporte[0])
        dia = int(f_reporte[1])
        fecha_r = datetime(anio,mes,dia)
        ndias = (fecha_r - fecha_e).days
    except:
        dias_exhi.append(0)
        ingreso.append(0)
        ingreso_dia.append(0)
        continue
    
    dias_exhi.append(ndias)
        
    dinero = df_paises.loc[ind, 'Box Office']
    aux1 = dinero.replace('.00','')
    aux2 = aux1.replace('$','')
    num = aux2.replace(',','')
    ing_estreno = int(num)
    ingreso.append(ing_estreno)
    
    por_dia = round(ing_estreno/ndias)
    ingreso_dia.append(por_dia)

paises_limpio = pd.DataFrame({'Territory':df_paises['Territory'], 'Release':fecha_estreno, 'Box-Office':ingreso,
                             'days_exhi':dias_exhi, 'income_day':ingreso_dia})
paises_limpio

In [ ]:
fecha_1 = fecha_ini
fechas = list()
fechas_str = list()
fecha_fin = hoy
ndias = (fecha_fin - fecha_ini).days

#Creación de la lista de fechas, la cual será el índice del dataframe
fechas.append(fecha_1)
f_str = fecha_1.strftime('%Y-%m-%d')
fechas_str.append(f_str)
for d in range(1000):
    fecha_1 = fecha_1 + timedelta(days = 1)
    fechas.append(fecha_1)
    f_str = fecha_1.strftime('%Y-%m-%d')
    fechas_str.append(f_str)
    if fecha_1.strftime('%Y-%m-%d') == fecha_fin.strftime('%Y-%m-%d'):
        break

df_serie = pd.DataFrame(index = fechas_str)  #fechas en el eje vertical 

for ind in paises_limpio.index:
    ingreso_pais = list()
    
    if paises_limpio['Box-Office'][ind] == 0:
        continue
    
    pais = paises_limpio['Territory'][ind]
    #Agregado de los países que ya estaban en la serie del dataframe anterior
    if pais in [*serie_previa] and serie_previa[pais].sum() != 0:
        ingreso_pais = list(serie_previa[pais])
        dif_ing = paises_limpio['Box-Office'][ind] - serie_previa[pais].sum()
        if dif_ing <= 1: #Al parecer, por algún ajuste en el reporte de cada país, aparecen números negativos 
            dif_ing = 0
        
        #Ciclo para encontrar la fecha del reporte previo
        i = 0
        for fecha in serie_previa.index:
            ingreso = serie_previa.loc[fecha,pais]
            i = i + 1
            if ingreso == 0 and serie_previa.loc[serie_previa.index[i-2],pais] > 0:
                reporte_previo = fecha
        
        reporte_pais = datetime.strptime(df_paises['Report Date'][ind], '%m/%d/%Y')
        reporte_previo_format = datetime.strptime(reporte_previo, '%Y-%m-%d')
        dif_dias = (hoy - reporte_previo_format).days
        
        if dif_dias == 0:
            por_dia = 0
        else:
            por_dia = round(dif_ing/dif_dias)
            rebanada = len(ingreso_pais) - dif_dias
            ingreso_pais = ingreso_pais[:rebanada]
        
        for fecha in fechas:
            if fecha.strftime('%Y-%m-%d') == hoy.strftime('%Y-%m-%d'):
                ingreso_pais.append(0)
                break
            if fecha >= reporte_previo_format:
                ingreso_pais.append(por_dia)
        
        reporte_pais = reporte_pais.strftime('%Y-%m-%d')

        try:
            df_serie[pais] = ingreso_pais
        except:
            df_serie[pais] = ingreso_pais[:-3]
        continue
    
    for fecha in fechas:
        print(pais, fecha)
        if (fecha != paises_limpio['Release'][ind] and 
            len(ingreso_pais) < (paises_limpio['Release'][ind] - fecha_ini).days):
            for i in range(100):
                if timedelta(days = i) == paises_limpio['Release'][ind] - fecha_ini:
                    break
                if len(ingreso_pais) == ndias + 1:  #Por si la fecha de estreno es posterior al día actual
                    break
                else:
                    ingreso_pais.append(0)
                    
        if fecha == paises_limpio.loc[ind, 'Release']:
            for d in range(100):
                if d >= paises_limpio.loc[ind, 'days_exhi']:
                    break
                ingreso_pais.append(paises_limpio['income_day'][ind])
                
        num_aux = (paises_limpio['Release'][ind] - fecha_ini).days + paises_limpio['days_exhi'][ind]
        
        if fecha != paises_limpio['Release'][ind] and len(ingreso_pais) >= num_aux:
            if len(ingreso_pais) == ndias:
                ingreso_pais.append(0) #Apilación extra consecuencia del día de reporte
                break
            ingreso_pais.append(0)
    print(pais, ingreso_pais)
                
    try:
        df_serie[pais] = ingreso_pais
    except:
        df_serie[pais] = ingreso_pais[:-1]
        
#Inserción de EUA y Canadá
le = list(serie_previa['United States'])
lc = list(serie_previa['Canada'])

i = 0
indi = 0
eua = list()
can = list()
for ing in le:
    if ing == 0:
        indi = indi + 1
        if indi == 3:
            eua.append(round(domestic*rate_eua))
            eua.append(0)
            can.append(round(domestic*rate_can))
            can.append(0)
            break
    i = i + 1
    eua.append(le[i-1])
    can.append(lc[i-1])
        
df_serie['United States'] = eua
df_serie['Canada'] = can
    
#Inserción del ingreso en el resto del mundo
l_mundo = list()
ndias = len(df_serie)-1
por_dia = resto_mundo/ndias
for fecha in df_serie.index:
    if fecha == df_serie.index[-1]:
        l_mundo.append(0)
        break
    l_mundo.append(round(por_dia))
    
df_serie['Rest of World'] = l_mundo

#Inserción de la suma por fecha
suma_fechas = list(df_serie.sum(axis = 1))
df_serie['Suma'] = suma_fechas
df_serie

df_serie

In [ ]:
suma_paises = pd.DataFrame(df_serie.sum(axis = 0), columns = ['Suma'])
suma_paises.drop(['Suma', 'Central America', 'Rest of World'], axis = 'index', inplace = True)
suma_paises.to_csv('suma_paises.csv')
df_serie.drop([df_serie.index[-1]], axis = 0, inplace = True)
df_serie.to_csv('serie_tiempo.csv')